In [101]:
#Benchmarking Variational Quantum State Diagonalization Algorithm

In [1]:
import pennylane as qml 
from pennylane import numpy as np
import matplotlib.pyplot as plt

In [2]:
# = 2
#n = 2*m
def test_prep(param):
    for i in range(2*len(param)):
        qml.Hadamard(i)
#    return [qml.expval(qml.PauliZ(i)) for i in range(2*m)]
drawer = qml.draw(test_prep)
print(drawer([0,1]))
#test_prep(m)

0: ──H─┤  
1: ──H─┤  
2: ──H─┤  
3: ──H─┤  


In [3]:
#TODO
#Make angles equal in the ith subsystem of registers 1 and 2
#param needs to be specifiable with m angles
#param = [0, 1, 2, 3]

def ansatz(param):
    for i in range(len(param)):
        qml.RZ(param[i], wires = i)
        qml.RZ(param[i], wires = i+len(param))
        qml.RX(np.pi/2, wires = i)
        qml.RX(np.pi/2, wires = i+len(param))
drawer = qml.draw(ansatz)
print(drawer([0,1]))

0: ──RZ(0.00)──RX(1.57)─┤  
2: ──RZ(0.00)──RX(1.57)─┤  
1: ──RZ(1.00)──RX(1.57)─┤  
3: ──RZ(1.00)──RX(1.57)─┤  


In [4]:
def cost_fun(param):
    for i in range(len(param)):
        qml.CNOT(wires = [i,i+len(param)])
drawer = qml.draw(cost_fun)
print(drawer([0,1]))

0: ─╭●─┤  
2: ─╰X─┤  
1: ─╭●─┤  
3: ─╰X─┤  


In [43]:
s = 1000
m = 2
n = 2*m
dev = qml.device("default.qubit", wires = 10, shots = s)
@qml.qnode(dev, interface = "autograd")
def circuit(param):
    test_prep(param)
    ansatz(param)
    cost_fun(param)
    return qml.probs(wires = [i for i in range(2*len(param))])

In [44]:
def VQSD(param):
    return -circuit(param)[0]

In [45]:
t = []
for i in range(5):
    t.append(0.2)
theta = np.array(t, requires_grad=True)
print(theta)

[0.2 0.2 0.2 0.2 0.2]


In [46]:
diag = [VQSD(theta)]
print(diag)

[-0.008]


In [47]:
# store the values of the circuit parameter
angle = [theta]
print(angle)
#len(angle)

[tensor([0.2, 0.2, 0.2, 0.2, 0.2], requires_grad=True)]


In [48]:
###To Do
#Figure out why the energy stays roughly constant

opt = qml.GradientDescentOptimizer(stepsize=0.2)
max_iterations = 1000
conv_tol = 1e-06

for n in range(max_iterations):
    theta, prev_diag = opt.step_and_cost(VQSD, theta)

    diag.append(VQSD(theta))
    angle.append(theta)

    conv = np.abs(-1 - diag[-1])

    if n % 2 == 0:
        print(f"Step = {n},  Diagonality = {diag[-1]:.8f} Ha")

    if conv <= conv_tol:
        break

#print("\n" f"Final value of the ground-state energy = {diag[-1]:.8f} Ha")
print("\n" f"Optimal value of the circuit parameter = {angle[-1]}")

Step = 0,  Diagonality = -0.00800000 Ha
Step = 2,  Diagonality = -0.00600000 Ha
Step = 4,  Diagonality = -0.01100000 Ha
Step = 6,  Diagonality = -0.01100000 Ha
Step = 8,  Diagonality = -0.00700000 Ha
Step = 10,  Diagonality = -0.00900000 Ha
Step = 12,  Diagonality = -0.00500000 Ha
Step = 14,  Diagonality = -0.00800000 Ha
Step = 16,  Diagonality = -0.00700000 Ha
Step = 18,  Diagonality = -0.01200000 Ha
Step = 20,  Diagonality = -0.01100000 Ha
Step = 22,  Diagonality = -0.01300000 Ha
Step = 24,  Diagonality = -0.01100000 Ha
Step = 26,  Diagonality = -0.01000000 Ha
Step = 28,  Diagonality = -0.01100000 Ha
Step = 30,  Diagonality = -0.01000000 Ha
Step = 32,  Diagonality = -0.00700000 Ha
Step = 34,  Diagonality = -0.01100000 Ha
Step = 36,  Diagonality = -0.00800000 Ha
Step = 38,  Diagonality = -0.00900000 Ha
Step = 40,  Diagonality = -0.01400000 Ha
Step = 42,  Diagonality = -0.01100000 Ha
Step = 44,  Diagonality = -0.01600000 Ha
Step = 46,  Diagonality = -0.01100000 Ha
Step = 48,  Diagonali

In [36]:
angle[-1]

tensor([1.5499, 1.5622, 1.5489, 1.5538], requires_grad=True)

In [32]:
diag

[0.059,
 0.074,
 0.065,
 0.06,
 0.061,
 0.06,
 0.06,
 0.047,
 0.073,
 0.062,
 0.061,
 0.052,
 0.061,
 0.055,
 0.062,
 0.063,
 0.058,
 0.051,
 0.052,
 0.086,
 0.072,
 0.06,
 0.057,
 0.051,
 0.057,
 0.08,
 0.063,
 0.068,
 0.067,
 0.067,
 0.069,
 0.057,
 0.06,
 0.065,
 0.07,
 0.059,
 0.067,
 0.067]

In [33]:
len(diag)

38

In [34]:
v = [0, 1, 2]
len(v)

3